In [ ]:
import datetime as dt
from io import BytesIO
from typing import NamedTuple

import pandas as pd

from micro_namedtuple_sqlite_persister.persister import Engine, register_serde


In [ ]:
def serialize_df(obj: pd.DataFrame) -> bytes:
    return obj.to_parquet()


def deserialize_df(data: bytes) -> pd.DataFrame:
    return pd.read_parquet(BytesIO(data))


register_serde(serialize_df, deserialize_df)


class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score_df: pd.DataFrame


In [ ]:
engine = Engine(":memory:")
engine.ensure_table_created(MyModel)

row = MyModel(None, "foo", dt.datetime.now(), pd.DataFrame({"a": [1, 2, 3]}))
row = engine.insert(row)
print(row.id)
# >>> 1


engine.insert(row._replace(score_df=pd.DataFrame({"a": [1, 2, 3, 4]})))
# >>> IdAlreadyExistsError


In [ ]:
engine.update(row._replace(score_df=pd.DataFrame({"a": [1, 2, 3, 4]})))
# success

engine.update(row._replace(id=None))
# IdMissingError

engine.update(row._replace(id=999))
# IdNotFoundError


In [ ]:
assert row.id is not None
engine.delete(MyModel, row.id)
# success